In [6]:
%matplotlib qt

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import numpy as np
import datetime

# User input for the year
year = input('Year = ')

# Read the CSV file into a DataFrame
df = pd.read_csv(f'{year}/scinti_S4cGPS{year}.csv', index_col=0, low_memory=False)

# Convert date and time columns to datetime format
df['DateTime_UTC'] = pd.to_datetime(df['DateTime_UTC'])
df['DateTime_IST'] = pd.to_datetime(df['DateTime_IST'])

# Set DateTime_UTC as the index
df.set_index('DateTime_UTC', inplace=True)

# Convert Total_S4_Sig1 column to float
df['Total_S4_Sig1'] = df['Total_S4_Sig1'].astype(float)
df['S4cor'] = df['S4cor'].astype(float)

# Filter rows where Total_S4_Sig1 <= 1.25
# df = df[df['Total_S4_Sig1'] <= 1.25]

# Calculate maximum S4 value for each DateTime_UTC
df['S4_max'] = df.groupby(['DateTime_UTC'])['S4cor'].transform(max)

# Remove duplicate DateTime_UTC entries, keeping the last occurrence
df = df[~df.index.duplicated(keep='last')]

# Set S4_max values less than 0.3 to NaN
df.loc[df['S4_max'] < 0.2, 'S4_max'] = np.nan

# Create a date range from January 1st to January 1st of the next year with a frequency of 1 minute
idx = pd.date_range(f'01-01-{year}', f'01-01-{int(year)+1}', freq='1min')

# Reindex the DataFrame with the new date range and fill missing values with NaN
df.index = pd.DatetimeIndex(df.index)
df = df.reindex(idx, fill_value=np.nan)

# Add 5.5 hours to the index to convert to IST (Indian Standard Time)
df['DateTime_IST'] = df.index + datetime.timedelta(hours=5.5)

# Pivot the DataFrame to create a heatmap
df_pivot = df.pivot_table(index=df.index.time, columns=df.index.date, values='S4_max', dropna=False)
df_pivot = df_pivot[df_pivot.index >= datetime.time(12, 30, 0)]

# Add 5.5 hours to the index of the pivoted DataFrame to convert to IST
delta = datetime.timedelta(hours=5.5)
datetime_index = pd.to_datetime(df_pivot.index, format='%H:%M:%S')
datetime_index += delta
new_time_index = datetime_index.time
df_pivot.index = new_time_index

# Create a heatmap using seaborn
fig, axes = plt.subplots(figsize=(21, 5))

sns.set(rc={'axes.linewidth': 2,
            'axes.facecolor': '#cdd3d4',
            'axes.edgecolor': 'black' ,
            'axes.labelcolor': 'black',
            'text.color': 'black', 
            'xtick.color': 'black', 
            'ytick.color': 'black',
            'xtick.major.width': 2, 
            'ytick.major.width': 2,
            'ytick.minor.width': 1 ,
            'xtick.minor.width': 1 ,
            'xtick.major.size': 6, 
            'ytick.major.size': 6, 
            'xtick.minor.size': 3, 
            'ytick.minor.size': 3,
            'xtick.direction': 'in',
            'ytick.direction': 'in',
            'font.size': 15,
            'font.family':'serif',
            'legend.fontsize':15,
            'legend.borderaxespad': 1.9, 
            'axes.labelpad': 10  }, 
        style='ticks', 
        context='paper', 
        font_scale=2)

res = sns.heatmap(df_pivot, cmap='jet', cbar_kws={'label': '$S_{4c}$ index'}, vmin=0.3, vmax=1.4, xticklabels=50, yticklabels=120)
axes.invert_yaxis()
axes.xaxis.set_major_formatter(mdates.DateFormatter('%j'))

plt.xlabel(f'Day of Year {year}')
plt.ylabel('Time (LT)')

for _, spine in res.spines.items():
    spine.set_visible(True)
    spine.set_linewidth(2)

# Define the dates for vertical lines
vertical_dates = ['2023-12-31','2023-12-14','2023-02-17','2023-03-03','2023-01-09','2023-08-05','2023-08-07','2023-01-06','2023-03-29','2023-06-20','2023-02-11','2023-01-10','2023-07-02','2023-11-28','2023-05-16','2023-05-20','2023-09-21','2023-02-28','2023-09-20','2023-05-03']

# Convert the dates to datetime objects
vertical_dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in vertical_dates]

# Plot vertical lines for the specified dates
for date in vertical_dates:
    plt.axvline(x=date, color='gray', linestyle='--', alpha=0.5)
    
    
given_dates = vertical_dates

# Convert given dates to datetime format
given_dates = pd.to_datetime(given_dates)

# ... (Your existing code)

# Create a heatmap using seaborn
fig, axes = plt.subplots(figsize=(21, 5))

# ... (Your existing code for styling)

# Draw vertical lines for given dates
for date in given_dates:
    axes.axvline(date, color='red', linestyle='--', alpha=0.5)  # Adjust color, linestyle, and alpha as needed

# ... (Your existing code)

# Display the plot or save it
plt.show()    




Year = 2023


In [1]:
!pwd

/media/bhuvi/Analysis/PhD_Processing/GPS/P1_fin_code
